<header style="padding:1px;background:#f9f9f9;border-top:3px solid #00b2b1"><img id="Teradata-logo" src="https://www.teradata.com/Teradata/Images/Rebrand/Teradata_logo-two_color.png" alt="Teradata" width="220" align="right" />

<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>Text Term Frequency Analysis - Python</b>
</header>

<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>Introduction</b></p>
<p style = 'font-size:16px;font-family:Arial'>
This demo will analyse the text in rows of the table to find the TF-IDF or Term Frequency-Inverse Document Frequency is an indicator of a term's importance in a specific document based on the entire corpus of documents.    
This is a demonstration of Vantage capabilities for functional demos e.g.
    <li style = 'font-size:16px;font-family:Arial'> NGramSplitter Function - tokenizes (splits) an input stream of text and outputs n multigrams (called n-grams) based on the specified Reset, Punctuation, and Delimiter syntax elements.</li>
</p>
<p style = 'font-size:16px;font-family:Arial'> This notebook demonstrate how the function is used in Python kernel, there is a similar notebook which shows the same features in sql kernel using  <a href = 'https://www.docs.teradata.com/r/Enterprise_IntelliFlex_VMware/Teradata-Package-for-Python-User-Guide-17.20/Introduction-to-Teradata-Package-for-Python'>TeradataML Package</a>,there is a similar notebook which shows the same features in sql kernel.More information on the functions can be found in <a href = 'https://docs.teradata.com/r/Enterprise/Teradata-Package-for-Python-Function-Reference-17.20'>TeradataML Python Reference</a> </p>
<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>Steps</b></p>
<p style = 'font-size:16px;font-family:Arial'>
    <li style = 'font-size:16px;font-family:Arial'> Connect to Vantage and read the dataset. </li>
    <li style = 'font-size:16px;font-family:Arial'> Use NGramSplitter SQL to create a table of grams of n-size. </li>
    <li style = 'font-size:16px;font-family:Arial'> Express SQL to calculate TF-IDF and store the output in a table. </li>
    <li style = 'font-size:16px;font-family:Arial'> Retrieve the data as a local dataframe. </li>
    <li style = 'font-size:16px;font-family:Arial'> Basic visualization to show top 30 important terms. </li>
</p>

<p style = 'font-size:16px;font-family:Arial'>Begin running steps with Shift + Enter keys.

<h1 style = 'font-size:28px;font-family:Arial;color:#E37C4D'><b>Import python packages, connect to Vantage and explore the dataset</b></h1>

In [ ]:
# TeradataML components used
from teradataml                     import func
from teradataml.analytics.sqle      import NGramSplitter
from teradataml.dataframe.dataframe import DataFrame, in_schema
#from teradataml.dataframe.copy_to   import copy_to_sql
from teradataml.context.context     import create_context, remove_context, get_context
from teradataml.options.display     import display
# DataTypes from TeradataSQLAlchemy
from teradatasqlalchemy.types import *

# Pandas/Numpy/matplotlib as Standas Pandas for Standard Dataframe
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# general purpose 
import getpass
import warnings
import sys

# set message level
display.print_sqlmr_query = False
warnings.filterwarnings('ignore')

<p style = 'font-size:16px;font-family:Arial'>You will be prompted to provide the password. Enter your password, press the Enter key, then use down arrow to go to next cell.

In [ ]:
%run -i ../startup.ipynb

<p style = 'font-size:16px;font-family:Arial'>Below command will make a connection to the Vantage environment and set (optional) QueryBand for database session.

In [ ]:
eng = create_context(host = 'host.docker.internal', username='demo_user', password = password)
print(eng)
eng.execute("SET query_band='DEMO=TF_IDF_Workflow_Pythonista.ipynb;' UPDATE FOR SESSION;")

<h2 style = 'font-size:18px;font-family:Arial;color:#E37C4D'> <b>Getting Data for This Demo</b></h2>
<p style = 'font-size:16px;font-family:Arial'>We have provided data for this demo on cloud storage. You have the option of either running the demo using foreign tables to access the data without using any storage on your environment or downloading the data to local storage which may yield somewhat faster execution, but there could be considerations of available storage. There are two statements in the following cell, and one is commented out. You may switch which mode you choose by changing the comment string.</p>

In [ ]:
%%time
%run -i ../run_procedure.py "call get_data('DEMO_Retail_cloud');"
# takes about 25 seconds, estimated space: 0 MB
#%run -i ../run_procedure.py "call get_data('DEMO_Retail_local');" 
# takes about 50 seconds, estimated space: 23 MB

<p style = 'font-size:16px;font-family:Arial'>Next is an optional step – if you want to see status of databases/tables created and space used.</p>

In [ ]:
# %run -i ../run_procedure.py "call space_report();"

<p style = 'font-size:16px;font-family:Arial'>Get the data from Vantage object <i>"DEMO_Retail"."Web_Comment"</i> in the DataFrame.

In [ ]:
%%time
tdf_reviews = DataFrame('"DEMO_Retail"."Web_Comment"')

<h2 style = 'font-size:18px;font-family:Arial;color:#E37C4D'> <b>Getting familiar with the dataset</b></h2>

In [ ]:
tdf_reviews.info()

In [ ]:
print("Object tdf_reviews has {} records and uses {} bytes in memory."
      .format(tdf_reviews.size, sys.getsizeof(tdf_reviews)));

<p style = 'font-size:16px;font-family:Arial'>Let us see how the data in the table looks like. We have taken one comment.

In [ ]:
# check an example comment
tdf_reviews[(tdf_reviews.comment_id == 30)]

<h1 style = 'font-size:28px;font-family:Arial;color:#E37C4D'><b>Use the NGram Splitter SQL Function</b></h1>
<p style = 'font-size:16px;font-family:Arial'>NGram function will split the corpus of documents into "terms" (grams) of selected size.  Specifically, this example will create a table called "tbl_grams" that is the result of splitting each "document" (review) into two-word chunks (grams).  Each row in this table includes;
<ol style = 'font-size:16px;font-family:Arial'>
    <li>The two-word chunk (ngram).</li>
    <li>The source review id (row_id).</li>
     <li>Chunk length (n).</li>
     <li>The count of this chunk in the review (frequency).</li>
     <li>The count of this chunk in all the reviews (totalcnt)</li>
</ol>
<p style = 'font-size:16px;font-family:Arial'>The splitting algorithm <a href = 'https://docs.teradata.com/r/Enterprise/Teradata-Package-for-Python-Function-Reference-17.20/teradataml-Analytic-Database-SQL-Engine-Analytic-Functions/Supported-on-Database-Versions-16.20.xx-17.00.xx-17.05.xx/NGramSplitter'>NGramSplitter </a>can be controlled with delimeters, punctuation indicators, etc.</p>

In [ ]:
tdf_grams = NGramSplitter(
               data             = tdf_reviews
              ,text_column      = 'comment_text'
              ,accumulate       = 'comment_id'
              ,grams            = "2"
              ,overlapping      = True
              ,to_lower_case    = True
              ,delimiter        = " "
              ,punctuation      = '[`~#^&*()-]'
              ,total_gram_count = True
            ).result

<p style = 'font-size:16px;font-family:Arial'>In the dataframe created above we can see the NGram function applied to the web comment column. We can see the frequency and the total number of times the ngram appear in the column.  
Let us check how the comment id we saw earlier looks after converting to ngrams

In [ ]:
# check an example comment
tdf_grams[(tdf_grams.comment_id == 30)]

In [ ]:
# check count of distinct CommentID (distinct_Comment should be 22641)
tdf_grams.assign(drop_columns=True
                ,distinct_Comment=tdf_grams.comment_id.distinct().count())

<h1 style = 'font-size:28px;font-family:Arial;color:#E37C4D'><b>Create the TF-IDF Table</b></h1>
<p style = 'font-size:16px;font-family:Arial'><a href = 'https://en.wikipedia.org/wiki/Tf%E2%80%93idf'<b>TF-IDF</b> or <b>Term Frequency-Inverse Document Frequency</b> </a> is an indicator of a term's <b>importance</b> in a specific document based on the entire corpus of documents.  This value is calculated by taking the Product of:
<ul style = 'font-size:16px;font-family:Arial'>
    <li>Term Frequency = (Number of Terms in the Document)/(Number of Terms in the Corpus)</li>
    <li>Inverse Document Frequency = Natural Log((Total Number of Documents)/(Number of Documents with the Term))</li>
 </ul>   
<p style = 'font-size:16px;font-family:Arial'>
This can be accomplished in teradatml.dataframe using the results table created using NGgram Splitter function in above step.
<br>
    
    tdf_tf_idf.tf     = tdf_grams.frequency / tdf_grams.totalcnt
    tdf_tf_idf.idf    = ln( (Total distinct tdf_grams.comment_id) / (Total distinct tdf_grams.ngram) )
    tdf_tf_idf.tf_idf = idf * tf


In [ ]:
#get Total distinct tdf_grams.comment_id --> iDistinctComment (SQL column: tot_docs)
iDistinctComment = tdf_grams.assign(drop_columns=True
                                         ,distinct_Comment = tdf_grams.comment_id.distinct().count()
                                   ).get_values()[0][0]

In [ ]:
# get Total distinct tdf_grams.ngram --> td_ngram_count (SQL column: tot_term)
td_ngram_count  = tdf_grams.groupby(["ngram"]).count().select(["ngram", "count_comment_id"])

In [ ]:
# first draft of tdf_tf_idf by selecting column to keep & adding iDistinctComment
tdf_tf_idf = tdf_grams.assign(drop_columns = True
                                   ,comment_id = tdf_grams.comment_id
                                   ,term      = tdf_grams.ngram
                                   ,tf         = tdf_grams.frequency.cast(type_=FLOAT) / tdf_grams.totalcnt 
                                   ,tot_docs   = (int(iDistinctComment))
                                   )

In [ ]:
# left-join tdf_tf_idf to td_ngram_count
tdf_tf_idf = tdf_tf_idf.join(other   = td_ngram_count
                            ,on      = ["term=ngram"]
                            ,how     = "left" 
                            ,lsuffix = "t1", rsuffix = "t2")

In [ ]:
# add inner division of idf-LN
tdf_tf_idf = tdf_tf_idf.assign(idf_    = (tdf_tf_idf.tot_docs.cast(type_=FLOAT) / tdf_tf_idf.count_comment_id) )

In [ ]:
# add idf
tdf_tf_idf = tdf_tf_idf.assign(idf     = func.ln(tdf_tf_idf.idf_.expression))

In [ ]:
# add tf_idf
tdf_tf_idf = tdf_tf_idf.assign(tf_idf  = tdf_tf_idf.idf * tdf_tf_idf.tf )

In [ ]:
# check whether all columns are now pulled together
tdf_tf_idf.info()

In [ ]:
%%time
# pull it local to pandas-dataframe
# again.. - Lazy Evaluation & this step does all the work, taking some time
df_tf_idf = tdf_tf_idf.to_pandas(all_rows = True)
df_tf_idf['tf_idf'] = df_tf_idf['tf_idf'].astype(float)

In [ ]:
# check mem sizes of teradataml.dataframe <-> pandas.dataframe
print("teradataml.dataframe tdf_tf_idf has {:>10} bytes in memory.".format(sys.getsizeof(tdf_tf_idf)));
print("pandas.dataframe     df_tf_idf  has {:>10} bytes in memory.".format(sys.getsizeof(df_tf_idf)));

<p style = 'font-size:16px;font-family:Arial'>Let us check the term frequency and inverse document frequency calculated for the comment we saw before

In [ ]:
# check an example comment
df_tf_idf[(df_tf_idf.comment_id == 30)]

<p style = 'font-size:16px;font-family:Arial'>Now, let us check the mostly used terms in our data

In [ ]:
df_tf_idf.sort_values(by = 'tf_idf', ascending = False)

<h1 style = 'font-size:28px;font-family:Arial;color:#E37C4D'><b>Visualize the Results</b></h1># Visualize the Results
<p style = 'font-size:16px;font-family:Arial'>Let's use Pandas and Matplotlib to do visualizations of the data:

In [ ]:
# get Top30 by tf_idf
df_top30 = df_tf_idf.sort_values(by = 'tf_idf', ascending = False).head(30)

In [ ]:
# plot it using build in plot
df_top30.sort_values(by = 'tf_idf', ascending = True)\
        .set_index('term')[['tf_idf']]\
        .plot(kind = 'barh', legend = True, figsize = (12, 9));

<p style = 'font-size:16px;font-family:Arial'>In this plot you see the top 30 terms which are used in the reviews. The plot is made up with pandas.dataframe build in <a href = 'https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.plot.htmlhttps://pandas.pydata.org/docs/reference/api/pandas.DataFrame.plot.html'>plot function </a> build on <a href = 'https://matplotlib.org/stable/api/indexhttps://matplotlib.org/stable/api/index'> matplolib </a>.

<h1 style = 'font-size:28px;font-family:Arial;color:#E37C4D'><b>Clean up </h1>
<h2 style = 'font-size:20px;font-family:Arial;color:#E37C4D'> <b>Database and Tables</b></h2>
<p style = 'font-size:16px;font-family:Arial'>The following code will clean up tables and databases created above.</p>

In [ ]:
%run -i ../run_procedure.py "call remove_data('DEMO_Retail');" 
#Takes 5 seconds

<h1 style = 'font-size:28px;font-family:Arial;color:#E37C4D'><b>Reference Links:</b></h1>
<ul style = 'font-size:16px;font-family:Arial'>
    <li><a href = 'https://docs.teradata.com/reader/eteIDCTX4O4IMvazRMypxQ/uDjppX7PJInABCckgu~KFg'>Teradata Python Package User Guide</a></li>
    <li><a href = 'https://docs.teradata.com/reader/GsM0pYRZl5Plqjdf9ixmdA/MzdO1q_t80M47qY5lyImOA'>Teradataml Python Reference</a></li>
    <li><a href = 'https://docs.teradata.com/r/Teradata-VantageTM-Analytics-Database-Analytic-Functions-17.20/Text-Analytic-Functions/NGramSplitter'>Teradata NGramSplitter Function Reference</a></li>
  
</ul>

<h1 style = 'font-size:28px;font-family:Arial;color:#E37C4D'><b>Credits</b></h1>
<p style = 'font-size:16px;font-family:Arial'>Thanks to Horst Hoeck of Teradata in Germany for contributing this "pure Python" version of the Text Term Frequency demonstration. </p>

<footer style="padding:10px;background:#f9f9f9;border-bottom:3px solid #394851">©2023 Teradata. All Rights Reserved</footer>